In [1]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)


In [2]:
tickers = ['EURUSD', 'GBPUSD', 'USDJPY', 'USDCAD', 'AUDUSD', 'USDCHF']
symbols_path = './backbone/data/backtest/symbols'
instruments = {}
df = pd.DataFrame()

In [3]:
for ticker in tickers:
    instruments[ticker] = pd.read_csv(os.path.join(symbols_path, f'{ticker}.csv'))
  
    instruments[ticker]['ticker'] = ticker
  
    print('Creando target')
    instruments[ticker] = instruments[ticker].sort_values(by='Date')
    instruments[ticker]['target'] = ((instruments[ticker]['Close'].shift(-1) - instruments[ticker]['Close']) / instruments[ticker]['Close']) * 100

    # bins = pd.IntervalIndex.from_tuples([(-100, -2), (-1, 1), (2, 100)])
    bins = [-100000, -0.1, 0.1, 100000]
    labels = [0, 1, 2]

    instruments[ticker]['target'] = pd.cut(instruments[ticker]['target'], bins, labels=labels)
    
    df = pd.concat([
        df,
        instruments[ticker]
    ])

    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:00:00')

    df = df.sort_values(by='Date')

Creando target
Creando target
Creando target
Creando target
Creando target
Creando target


In [4]:
df

,Date,Open,High,Low,Close,Volume,ema_12,ema_26,ema_50,ema_200,rsi,upper_bband,middle_bband,lower_bband,macd,macdsignal,macdhist,macdhist_yesterday,macd_flag,change_percent_ch,change_percent_co,change_percent_cl,change_percent_1_day,change_percent_2_day,change_percent_3_day,change_percent_h,change_percent_o,change_percent_l,ticker,target
0,2023-06-13 07:00:00,1.07791,1.07830,1.07762,1.07822,1054,1.076724,1.076224,1.075743,1.072892,66.703174,1.078846,1.077272,1.075698,0.000500,0.000235,0.000266,2.197159e-04,0,-0.01,0.03,0.06,0.03,0.07,0.14,0.14,0.17,0.16,EURUSD,0
0,2023-06-13 07:00:00,1.33563,1.33635,1.33547,1.33632,1511,1.336384,1.335895,1.335703,1.340664,52.974767,1.337616,1.336546,1.335476,0.000489,0.000564,-0.000075,-4.761812e-05,0,-0.00,0.05,0.06,0.05,-0.06,-0.05,-0.06,-0.07,-0.06,USDCAD,0
0,2023-06-13 07:00:00,139.39200,139.51500,139.38400,139.49100,2883,139.509662,139.496983,139.473759,139.487535,49.800525,139.675875,139.499600,139.323325,0.012679,0.032296,-0.019618,-2.160762e-02,0,-0.02,0.07,0.08,0.07,0.04,-0.04,-0.10,-0.18,-0.09,USDJPY,0
0,2023-06-13 07:00:00,0.90716,0.90753,0.90680,0.90680,542,0.907858,0.906996,0.905893,0.906453,49.075021,0.909297,0.907778,0.906259,0.000862,0.001263,-0.000400,-3.236656e-04,0,-0.08,-0.04,0.00,-0.04,-0.10,-0.16,-0.15,-0.19,-0.15,USDCHF,0
0,2023-06-13 07:00:00,1.25333,1.25336,1.25280,1.25315,2990,1.252350,1.253170,1.253390,1.248404,49.572493,1.253820,1.252486,1.251152,-0.000820,-0.001126,0.000306,2.352628e-04,0,-0.02,-0.01,0.03,-0.01,0.06,0.11,0.09,0.13,0.11,GBPUSD,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5775,2024-05-17 00:00:00,155.36300,155.38100,155.32100,155.35500,301,155.188002,155.077867,155.224804,155.430998,58.301223,155.449833,155.341800,155.233767,0.110134,0.013907,0.096227,1.120916e-01,0,-0.02,-0.01,0.02,-0.00,-0.04,0.02,0.03,0.07,0.05,USDJPY,NaN
5774,2024-05-17 00:00:00,1.08662,1.08676,1.08593,1.08654,398,1.086922,1.086778,1.085603,1.080153,47.998368,1.087258,1.086804,1.086350,0.000144,0.000361,-0.000217,-2.203638e-04,0,-0.02,-0.01,0.06,-0.01,-0.01,-0.05,-0.04,-0.04,-0.06,EURUSD,NaN
5775,2024-05-17 00:00:00,1.26688,1.26713,1.26618,1.26688,224,1.267024,1.266605,1.264760,1.257816,52.247093,1.267718,1.267120,1.266522,0.000419,0.000657,-0.000237,-2.481481e-04,0,-0.02,0.00,0.06,0.00,-0.00,-0.05,-0.04,-0.04,-0.04,GBPUSD,NaN
5775,2024-05-17 00:00:00,1.36151,1.36157,1.36043,1.36144,322,1.361707,1.361845,1.362569,1.366164,47.047894,1.361796,1.361500,1.361204,-0.000138,-0.000126,-0.000012,9.235840e-07,-1,-0.01,-0.01,0.07,-0.01,-0.02,0.01,-0.04,0.00,-0.06,USDCAD,NaN


In [5]:
df.target.value_counts()

target
0     28071
1      3312
-1     3266
Name: count, dtype: int64

In [15]:
train = df[(df['Date']>'2023-06-13 07:00:00') & (df['Date']<'2023-07-13 07:00:00')]
train

,Date,Open,High,Low,Close,Volume,ema_12,ema_26,ema_50,ema_200,rsi,upper_bband,middle_bband,lower_bband,macd,macdsignal,macdhist,macdhist_yesterday,macd_flag,change_percent_ch,change_percent_co,change_percent_cl,change_percent_1_day,change_percent_2_day,change_percent_3_day,change_percent_h,change_percent_o,change_percent_l,ticker,target
1,2023-06-13 08:00:00,1.33632,1.33690,1.33622,1.33659,1513,1.336415,1.335946,1.335737,1.340623,54.591282,1.337606,1.336538,1.335470,0.000469,0.000545,-0.000075,-0.000075,0,-0.02,0.02,0.03,0.02,0.07,-0.04,-0.07,-0.05,-0.06,USDCAD,0
1,2023-06-13 08:00:00,0.67618,0.67666,0.67586,0.67598,2146,0.675464,0.675078,0.674019,0.665777,58.707029,0.677129,0.675606,0.674083,0.000385,0.000377,0.000008,-0.000012,1,-0.10,-0.03,0.02,-0.03,-0.07,0.20,0.24,0.21,0.31,AUDUSD,0
1,2023-06-13 08:00:00,1.07822,1.07961,1.07815,1.07829,2582,1.076965,1.076377,1.075843,1.072945,67.084705,1.078864,1.077720,1.076576,0.000588,0.000305,0.000283,0.000266,0,-0.12,0.01,0.01,0.01,0.04,0.08,0.19,0.14,0.14,EURUSD,0
1,2023-06-13 08:00:00,139.49100,139.56700,139.37900,139.48100,3177,139.505252,139.495799,139.474043,139.487470,49.360653,139.569067,139.467000,139.364933,0.009453,0.027728,-0.018275,-0.019618,0,-0.06,-0.01,0.07,-0.01,0.06,0.04,0.01,-0.04,-0.02,USDJPY,0
1,2023-06-13 08:00:00,1.25315,1.25397,1.25282,1.25301,3840,1.252451,1.253158,1.253375,1.248450,48.682318,1.253883,1.252738,1.251593,-0.000707,-0.001042,0.000335,0.000306,0,-0.08,-0.01,0.02,-0.01,-0.03,0.05,0.11,0.11,0.09,GBPUSD,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,2023-07-13 06:00:00,138.51000,138.53600,138.39400,138.50300,3818,138.549994,138.919453,139.594916,141.840210,33.914124,138.633479,138.475600,138.317721,-0.369458,-0.436683,0.067225,0.059259,0,-0.02,-0.01,0.08,-0.01,0.00,-0.03,-0.03,0.14,0.24,USDJPY,0
527,2023-07-13 06:00:00,1.11460,1.11488,1.11424,1.11457,1099,1.112928,1.109833,1.106153,1.097105,78.495607,1.114941,1.114298,1.113655,0.003095,0.003093,0.000002,0.000084,0,-0.03,-0.00,0.03,-0.00,0.01,0.07,0.03,0.05,0.04,EURUSD,0
527,2023-07-13 06:00:00,1.31823,1.31833,1.31743,1.31769,1447,1.318633,1.319794,1.321786,1.325181,36.834421,1.319142,1.318348,1.317554,-0.001161,-0.001300,0.000139,0.000177,0,-0.05,-0.04,0.02,-0.04,-0.09,-0.07,-0.03,-0.01,-0.03,USDCAD,0
527,2023-07-13 06:00:00,1.30133,1.30186,1.30097,1.30117,1860,1.299376,1.297407,1.294225,1.282455,67.900292,1.302033,1.300280,1.298527,0.001969,0.001917,0.000051,0.000039,0,-0.05,-0.01,0.02,-0.01,0.07,0.16,0.14,0.14,0.15,GBPUSD,0


In [16]:
test = df[(df['Date']>'2023-07-13 07:00:00') & (df['Date']<'2023-07-13 21:00:00')]
test

,Date,Open,High,Low,Close,Volume,ema_12,ema_26,ema_50,ema_200,rsi,upper_bband,middle_bband,lower_bband,macd,macdsignal,macdhist,macdhist_yesterday,macd_flag,change_percent_ch,change_percent_co,change_percent_cl,change_percent_1_day,change_percent_2_day,change_percent_3_day,change_percent_h,change_percent_o,change_percent_l,ticker,target
529,2023-07-13 08:00:00,0.68105,0.68199,0.68057,0.68060,3182,0.679433,0.676906,0.673969,0.669232,68.799582,0.681746,0.680730,0.679714,0.002527,0.002543,-0.000016,0.000063,-1,-0.20,-0.07,0.00,-0.07,-0.13,0.01,0.19,0.16,0.22,AUDUSD,1
529,2023-07-13 08:00:00,138.52100,138.81800,138.52000,138.80600,4808,138.585605,138.883720,139.523515,141.777321,46.477923,138.806519,138.568000,138.329481,-0.298115,-0.394205,0.096090,0.073823,0,-0.01,0.21,0.21,0.21,0.22,0.21,0.12,0.02,0.06,USDJPY,-1
529,2023-07-13 08:00:00,1.31767,1.31855,1.31709,1.31849,2694,1.318486,1.319552,1.321502,1.325040,42.794748,1.319105,1.318184,1.317263,-0.001066,-0.001230,0.000163,0.000118,0,-0.00,0.06,0.11,0.06,0.06,0.02,-0.06,-0.09,-0.08,USDCAD,-1
529,2023-07-13 08:00:00,1.11411,1.11433,1.11296,1.11311,1885,1.113110,1.110369,1.106725,1.097432,66.647565,1.115298,1.114176,1.113054,0.002741,0.003001,-0.000260,-0.000106,0,-0.11,-0.09,0.01,-0.09,-0.13,-0.13,-0.04,-0.03,-0.09,EURUSD,1
529,2023-07-13 08:00:00,1.30063,1.30091,1.29872,1.29900,3240,1.299481,1.297746,1.294653,1.282798,56.082270,1.302146,1.300476,1.298806,0.001735,0.001882,-0.000147,0.000005,-1,-0.15,-0.13,0.02,-0.13,-0.17,-0.18,-0.04,0.03,-0.06,GBPUSD,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,2023-07-13 20:00:00,1.12058,1.12185,1.12023,1.12158,2430,1.118314,1.115317,1.111151,1.099773,83.979946,1.121840,1.120094,1.118348,0.002998,0.002856,0.000142,0.000085,0,-0.02,0.09,0.12,0.09,0.21,0.18,0.20,0.10,0.19,EURUSD,0
541,2023-07-13 20:00:00,1.31192,1.31271,1.31104,1.31115,2545,1.313910,1.316027,1.318612,1.323803,28.409389,1.313438,1.311952,1.310466,-0.002117,-0.001748,-0.000369,-0.000349,0,-0.12,-0.06,0.01,-0.06,-0.13,0.00,-0.13,-0.06,0.03,USDCAD,0
541,2023-07-13 20:00:00,0.68781,0.68866,0.68749,0.68860,1737,0.686235,0.682909,0.678808,0.671184,73.900437,0.690008,0.688376,0.686744,0.003326,0.003231,0.000095,0.000144,0,-0.01,0.11,0.16,0.12,0.21,-0.12,-0.11,-0.16,-0.03,AUDUSD,0
541,2023-07-13 20:00:00,1.31137,1.31230,1.31089,1.31210,3711,1.308003,1.304212,1.299770,1.285618,75.095768,1.312768,1.311062,1.309356,0.003791,0.003202,0.000589,0.000622,0,-0.02,0.06,0.09,0.05,0.20,0.05,0.07,0.04,0.14,GBPUSD,0


In [17]:
train[(train['target']==1)].shape

(332, 30)

In [18]:
train[(train['target']==0)].shape

(2490, 30)

In [19]:
train[(train['target']==1)].shape

(332, 30)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from backbone.probability_transformer import ProbabilityTransformer 
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from backbone.utils import load_function
from typing import Tuple
import yaml

In [21]:
scaler = StandardScaler()
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model = RandomForestClassifier()

pipe = Pipeline([
    ('scaler', scaler),
    ('prob_transf', ProbabilityTransformer(model)),
    ('log_reg', log_reg)
])

In [22]:
with open('configs/model_config.yml', 'r') as file:
    model_configs = yaml.safe_load(file)
param_grid = model_configs['random_forest']['param_grid']


In [23]:
n_splits = 2
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

search = GridSearchCV(
    pipe,
    param_grid,
    n_jobs=-1,
    cv=stratified_kfold,
    scoring=make_scorer(f1_score, average='weighted')
)

# Tunear hiperparametros
search.fit(train.drop(columns=['target', 'Date', 'ticker']), train.target)



GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('prob_transf',
                                        ProbabilityTransformer(model=RandomForestClassifier())),
                                       ('log_reg',
                                        LogisticRegression(max_iter=1000,
                                                           multi_class='multinomial'))]),
             n_jobs=-1,
             param_grid={'prob_transf__model__max_depth': [20, 25, 30, 35, 40],
                         'prob_transf__model__min_samples_leaf': [1],
                         'prob_transf__model__min_samples_split': [2],
                         'prob_transf__model__n_estimators': [50, 60, 70, 80,
                                                              90, 100, 110,
                                                              120],
                         'prob_transf__model__random_state': [42]},
             scoring=make_scorer(f1_score, average=weighted))

In [24]:
pipeline = search.best_estimator_


In [29]:
proba = pipeline.predict_proba(test.drop(columns=['target', 'Date', 'ticker']))

In [31]:
proba[0].max()


0.7886113895845857

In [32]:
proba[0].argmax()

2

In [27]:
test.target

529    1
529   -1
529   -1
529    1
529    1
      ..
541    0
541    0
541    0
541    0
541    0
Name: target, Length: 78, dtype: category
Categories (3, int64): [-1 < 0 < 1]